In [55]:
pip install pdfplumber

In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
 # data processing, CSV file I/O (e.g. pd.read_csv)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [57]:
import pandas as pd
# import seaborn as sns
df = pd.read_csv('/content/items_en.csv')
df.dropna(subset=['description'], inplace=True)#drop all rows with null descriptions
print(df.isnull().sum())

vectorizer = TfidfVectorizer()#Globally defined vectorizer




item_id          0
language         0
name             0
nb_views        38
description      0
created_at       0
Difficulty     540
Job              0
Software         0
Theme            0
duration         0
type             0
dtype: int64


In [58]:
#dropping irrelevant colomns
df = df.drop(columns=['language','created_at','Software','Theme','duration','item_id','nb_views','Job','type','Difficulty'])


df.head()


,name,description
0,What is OneDrive for Business?,OneDrive for Businessis an online libraryto st...
1,"Add, restore, delete documents on OneDrive","To create a new document in OneDriveclick New,..."
2,Share documents,"By default, all content is private, so you don..."
3,Recycle bin,"When you delete a file or afolder, OneDrive gi..."
4,Sync OneDrive for Business to your computer (W...,The one drive synchronization application is a...


# Functions Definitons:


**ReadPDF to Text**




In [59]:
import pdfplumber
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stop = stopwords.words('english')
def pdf2text(file_path):
    '''Iterate over pages and extract text'''
    text = ''
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Cleaning csv**

In [60]:
def preprocess_text(text, name):
    # lowercasing
    lowercased_text = text.lower()

    # cleaning
    import re
    remove_punctuation = re.sub(r'[^\w\s]', '', lowercased_text)
    remove_white_space = remove_punctuation.strip()

    # Tokenization = Breaking down each sentence into an array
    from nltk.tokenize import word_tokenize
    tokenized_text = word_tokenize(remove_white_space)

    # Stop Words/filtering = Removing irrelevant words
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words('english'))
    stopwords_removed = [word for word in tokenized_text if word not in stopwords]

    # Stemming = Transforming words into their base form
    from nltk.stem import PorterStemmer
    ps = PorterStemmer()
    stemmed_text = [ps.stem(word) for word in stopwords_removed]

    # Putting all the results into a dataframe
    data_frame = pd.DataFrame({
        'NAME': [name],
        'DOCUMENT': [text],
        #'LOWERCASE': [lowercased_text],
        #'CLEANING': [remove_white_space],
        #'TOKENIZATION': [tokenized_text],
        ###'STOP-WORDS': [stopwords_removed],
        'PREPROCESSED': [' '.join(stemmed_text)]  # Store as string
    })

    return data_frame#returns single row ["name","document","stemming"]

def preprocessing(corpus):
    # Create an empty DataFrame
    data_frame = pd.DataFrame(columns=['NAME', 'DOCUMENT',  'PREPROCESSED'])#LOWERCASE', 'CLEANING', 'TOKENIZATION', 'STOP-WORDS',

    # Running preprocessing one by one
    for index, row in corpus.iterrows():
        preprocessed_row = preprocess_text( row['description'],row['name'])
        data_frame = pd.concat([data_frame,preprocessed_row], ignore_index=True)

    return data_frame


# **TF-IDF Calculation whole**

In [61]:
def calculate_tfidf(corpus):#caculate TF-IDF for dataframe
    # global vectorizer#acess  the globally defined vectorizer

    # Convert stemming words to string for vectorization
    pre_processed_docs = corpus['PREPROCESSED']
    #corpus=corpus.apply(' '.join)
    vectorizer = TfidfVectorizer()

    tfidf_matrix = vectorizer.fit_transform(pre_processed_docs)

    # Retrieve feature names (unique words) learned by tf-idf vectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Combine feature names above and weights -> array -> data frame
    df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Separate the name column
    df_tfidf['NAME'] = corpus['NAME'].values
    df_tfidf['DOCUMENT'] = corpus['DOCUMENT'].values
    df_tfidf['PREPROCESSED'] = corpus['PREPROCESSED'].values



    return df_tfidf


# **NLP**

**preprocessing of dataset description**

In [62]:

dataset=preprocessing(df)
df

,name,description
0,What is OneDrive for Business?,OneDrive for Businessis an online libraryto st...
1,"Add, restore, delete documents on OneDrive","To create a new document in OneDriveclick New,..."
2,Share documents,"By default, all content is private, so you don..."
3,Recycle bin,"When you delete a file or afolder, OneDrive gi..."
4,Sync OneDrive for Business to your computer (W...,The one drive synchronization application is a...
...,...,...
1132,Imbrication OneNote / Outlook,See who is collaborating.Identify and authenti...
1133,Imbrication OneNote / Teams,One note lets you automatically convertan outl...
1134,Turning web pages into notes,Password protect your notes.Password protectio...
1135,Organize your items with tags,"Imbrication OneNote, outlook.You can use OneNo..."


In [63]:
dataset.head()

,NAME,DOCUMENT,PREPROCESSED
0,What is OneDrive for Business?,OneDrive for Businessis an online libraryto st...,onedr businessi onlin libraryto store organ al...
1,"Add, restore, delete documents on OneDrive","To create a new document in OneDriveclick New,...",creat new document onedriveclick new document ...
2,Share documents,"By default, all content is private, so you don...",default content privat dont worri secur share ...
3,Recycle bin,"When you delete a file or afolder, OneDrive gi...",delet file afold onedr give youa chanc undo de...
4,Sync OneDrive for Business to your computer (W...,The one drive synchronization application is a...,one drive synchron applic avail offic 2016 dow...


In [64]:
#print(df.loc[0, 'description'])
tf_idf=calculate_tfidf(dataset)
tf_idf.head()

,00,000011000,000014000or,01,01a,01and,01it,01then,02,05,...,zwei,zweit,zwischen,último,único,über,übermitteln,NAME,DOCUMENT,PREPROCESSED
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,What is OneDrive for Business?,OneDrive for Businessis an online libraryto st...,onedr businessi onlin libraryto store organ al...
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Add, restore, delete documents on OneDrive","To create a new document in OneDriveclick New,...",creat new document onedriveclick new document ...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Share documents,"By default, all content is private, so you don...",default content privat dont worri secur share ...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Recycle bin,"When you delete a file or afolder, OneDrive gi...",delet file afold onedr give youa chanc undo de...
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sync OneDrive for Business to your computer (W...,The one drive synchronization application is a...,one drive synchron applic avail offic 2016 dow...


In [65]:
non_zero_counts = tf_idf.astype(bool).sum(axis=0)
non_zero_counts#gets values where TF-IDF are not zeros


00                 1
000011000          1
000014000or        1
01                 5
01a                1
                ... 
über               1
übermitteln        1
NAME            1009
DOCUMENT        1009
PREPROCESSED    1009
Length: 35582, dtype: int64

# **GetRecommendation**

In [66]:


def get_recommendations(name, tf_idf):
    from sklearn.metrics.pairwise import linear_kernel
    # Compute the cosine similarity matrix: dot product
    cosine_sim = linear_kernel(tf_idf.iloc[:,:-3], tf_idf.iloc[:,:-3])
    #maps between name & index
    indices = pd.Series(tf_idf.index, index=tf_idf['NAME'])
    idx = indices[name] #index of record
    # Get the pairwsie similarity scores of all courses with that course
    sim_scores = list(enumerate(cosine_sim[idx]))
    #sort courses based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # get score of most 10 similar courses
    sim_scores = sim_scores[1:11]
    #creates new list containg
    course_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar courses
    return df['name'].iloc[course_indices]


df.head()
# indices = pd.Series(df2.index, index=df2['title'])

,name,description
0,What is OneDrive for Business?,OneDrive for Businessis an online libraryto st...
1,"Add, restore, delete documents on OneDrive","To create a new document in OneDriveclick New,..."
2,Share documents,"By default, all content is private, so you don..."
3,Recycle bin,"When you delete a file or afolder, OneDrive gi..."
4,Sync OneDrive for Business to your computer (W...,The one drive synchronization application is a...


In [67]:
get_recommendations('Recycle bin',tf_idf)

944                                Recover deleted items
969    Recovering deleted items (Recycle bin and Seco...
328                                Recover deleted items
481                          Work anywhere with OneDrive
425                                     Editing of Items
848           Save Documents in OneDrive from Other Apps
482                           Manage files with OneDrive
426                           Moving and Copying Content
775          OneDrive - Discover the online storage tool
432                Using Files in a Synchronized Library
Name: name, dtype: object

In [68]:
#save last 3 csv and download
last_3_columns_df = tf_idf.iloc[:, -3:]
# last_3_columns_df
last_3_columns_df.to_csv('last_3_columns_tfidf.csv', index=False)



In [ ]:
# !pip install pdfplumber
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [ ]:

# file_path = '/content/example.pdf'
# extracted_text = pdf2text(file_path)
# print(extracted_text)


In [71]:
# #corpus is a large and structured set of texts
# corpus = [extracted_text]
# #initialize vectorizer
# vectorizer = TfidfVectorizer(stop_words=stop)
# #apply transform corpus to TF-IDF matrix
# pdf_matrix = vectorizer.fit_transform(corpus)
# # vectorizer.fit(corpus)
# #extract feature names : identified words like data science etc...
# tfidf_tokens = vectorizer.get_feature_names_out()
# print("TF-IDF pdf matrix:\n",pdf_matrix)
# print("Features extracted (tokens):\n", tfidf_tokens)